In [1]:
from etl import store_raw_data, clear_raw_data, get_raw_data, clear_clean_data, save_clean_data, get_clean_data, get_data

/opt/conda/lib/python3.11/site-packages/palmerpenguins/penguins.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
clear_raw_data()
print("test")

cleaning raw data
Trying to connect to database
After connection
After drop
✅ Table 'raw_data' deleted.
🔒 MySQL connection closed
table deleted
test


In [3]:
store_raw_data()

  species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0  Adelie  Torgersen            39.1           18.7              181.0   
1  Adelie  Torgersen            39.5           17.4              186.0   
2  Adelie  Torgersen            40.3           18.0              195.0   
3  Adelie  Torgersen             NaN            NaN                NaN   
4  Adelie  Torgersen            36.7           19.3              193.0   

   body_mass_g     sex  year  
0       3750.0    male  2007  
1       3800.0  female  2007  
2       3250.0  female  2007  
3          NaN     NaN  2007  
4       3450.0  female  2007  
columns_str species VARCHAR(255), island VARCHAR(255), bill_length_mm VARCHAR(255), bill_depth_mm VARCHAR(255), flipper_length_mm VARCHAR(255), body_mass_g VARCHAR(255), sex VARCHAR(255), year VARCHAR(255)
🔒 MySQL connection closed
rows_clean [['Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'male', 2007], ['Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'female'

In [4]:
clear_clean_data()

Trying to connect to database
After connection
After drop
✅ Table 'clean_data' deleted.
🔒 MySQL connection closed


In [5]:
save_clean_data()

🔒 MySQL connection closed
  species     island bill_length_mm bill_depth_mm flipper_length_mm  \
0  Adelie  Torgersen           39.1          18.7             181.0   
1  Adelie  Torgersen           39.5          17.4             186.0   
2  Adelie  Torgersen           40.3          18.0             195.0   
3  Adelie  Torgersen           None          None              None   
4  Adelie  Torgersen           36.7          19.3             193.0   

  body_mass_g     sex  year  
0      3750.0    male  2007  
1      3800.0  female  2007  
2      3250.0  female  2007  
3        None    None  2007  
4      3450.0  female  2007  
Null values species              0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
year                 0
island_Dream         0
island_Torgersen     0
sex_female           0
sex_male             0
dtype: int64
####################################################################################################
Data types 

In [6]:
from sklearn.model_selection import GridSearchCV, train_test_split
import mlflow
from sklearn.ensemble import RandomForestClassifier

In [7]:
mlflow.set_tracking_uri("http://mlflow:8003")

In [8]:
mlflow.set_experiment("random_forest_experiment")

<Experiment: artifact_location='s3://mlflows3/artifacts/1', creation_time=1758383471463, experiment_id='1', last_update_time=1758383471463, lifecycle_stage='active', name='random_forest_experiment', tags={}>

In [23]:
mlflow.autolog(log_input_examples= True, log_model_signatures = True, log_models = True, log_datasets = True,
               log_traces = True, disable = False, exclusive = False, disable_for_unsupported_versions = False,
               silent = False)

TypeError: autolog() got an unexpected keyword argument 'max_results'

In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
print(mlflow.__version__)

3.4.0


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
params = {
  "n_estimators": [33, 66, 200],
  "max_depth": [2, 4, 6],
  "max_features": [3, 4, 5]
}

rf = RandomForestClassifier()
searcher = GridSearchCV(estimator=rf, param_grid=params)

X, y = get_clean_data()

X_train, X_test, y_train, y_test = train_test_split(X, y)

with mlflow.start_run(run_name="autolog_with_grid_search") as run:
    searcher.fit(X_train, y_train)
    best = searcher.best_estimator_
    test_score = best.score(X_test, y_test)
    mlflow.log_metric("test_score", float(test_score))
    sig = infer_signature(X_train, best.predict(X_train))
    input_ex = X_train[:5]

    # 💡 Register directly here by giving a registered model name
    result = mlflow.sklearn.log_model(
        sk_model=best,
        artifact_path="model",
        registered_model_name="random-forest-regressor",  # <-- your model name
        signature=sig,
        input_example=input_ex,
    )
    version = result.registered_model_version  # string like "7"

client = MlflowClient()
# client.set_registered_model_alias("random-forest-regressor", "staging", version)
# when ready:
client.set_registered_model_alias("random-forest-regressor", "prod", version)
    

🔒 MySQL connection closed
After cleaning
        bill_length_mm        bill_depth_mm    flipper_length_mm  \
0  -0.8870812256081303   0.7854492273303183   -1.418346649339451   
1  -0.8134939891378182  0.12618789738684139  -1.0622502214477376   
2   -0.666319516197194  0.43046235736075433  -0.4212766512426535   
3  -1.3286046444300041   1.0897236873042313  -0.5637152223993389   
4  -0.8502876073729749   1.7489850172477082  -0.7773730791343669   

           body_mass_g island_Dream island_Torgersen sex_female sex_male  
0  -0.5641420770995512            0                1          0        1  
1  -0.5017030510904746            0                1          1        0  
2  -1.1885323371903167            0                1          1        0  
3  -0.9387762331540105            0                1          1        0  
4  -0.6890201291177043            0                1          0        1  
0    0
1    0
2    0
3    0
4    0
Name: species, dtype: int64
(342, 8)
(342,)
#####################

2025/09/22 03:42:20 INFO mlflow.sklearn.utils: Logging the 5 best runs, 22 runs will be omitted.


🏃 View run enthused-hawk-82 at: http://mlflow:8003/#/experiments/1/runs/ba8960ff33e64c9ba0faece475fda0d6
🧪 View experiment at: http://mlflow:8003/#/experiments/1
🏃 View run fun-bear-596 at: http://mlflow:8003/#/experiments/1/runs/517ecf0bf7c7462f9905a998310ae1f7
🧪 View experiment at: http://mlflow:8003/#/experiments/1
🏃 View run melodic-fish-32 at: http://mlflow:8003/#/experiments/1/runs/b04246a1765f41b1b9a77b86240f0032
🧪 View experiment at: http://mlflow:8003/#/experiments/1
🏃 View run able-fowl-269 at: http://mlflow:8003/#/experiments/1/runs/d88da2cd3e0d45eaa936a435bcc88246
🧪 View experiment at: http://mlflow:8003/#/experiments/1
🏃 View run bald-crane-836 at: http://mlflow:8003/#/experiments/1/runs/8f7512b8c0eb436894ae8a4f13d8fa3b
🧪 View experiment at: http://mlflow:8003/#/experiments/1


2025/09/22 03:42:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'random-forest-regressor'.
2025/09/22 03:42:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest-regressor, version 1
Created version '1' of model 'random-forest-regressor'.


🏃 View run autolog_with_grid_search at: http://mlflow:8003/#/experiments/1/runs/8ab31488e4eb4886b14934408a141c30
🧪 View experiment at: http://mlflow:8003/#/experiments/1


In [9]:
# match self.model_name:
#   case ModelType.RANDOM_FOREST:
#     self.model = RandomForestClassifier(n_estimators=300, max_depth=None, random_state=42)
#   case ModelType.SVM:
#     self.model = SVC(kernel='rbf', C=1.0, gamma='scale', class_weight='balanced', random_state=self.random_state)
#   case ModelType.NEURAL_NETWORK:
#     print("Training neural network")
#     self.model = MLPClassifier(
#         hidden_layer_sizes=(50,),
#         activation='tanh',
#         solver='lbfgs',
#         alpha=1e-4,
#         max_iter=1000,
#         random_state=self.random_state
#     )
#   case _:
#     raise ValueError(f"Model type {self.model_name} not supported")

In [3]:
model = mlflow.sklearn.load_model("models:/random-forest-regressor@prod")

# Use it like a normal sklearn estimator
y_pred = model.predict(X_test)
print(len(X_test))
print(y_pred)

NameError: name 'X_test' is not defined

In [42]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

for rm in client.search_registered_models():
    print("Model:", rm.name)
    for v in rm.latest_versions:
        print(f"  - version {v.version}, aliases={v.aliases}, run_id={v.run_id}")

Model: random-forest-regressor
  - version 1, aliases=[], run_id=8ab31488e4eb4886b14934408a141c30
Model: RandomForestProd
  - version 2, aliases=[], run_id=
